In [ ]:
!pip install -q efficientnet

In [ ]:
# loading packages

import os
import re
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf

from kaggle_datasets import KaggleDatasets
import efficientnet.tfkeras as efn
from tensorflow.keras import backend as K
import tensorflow_addons as tfa

In [ ]:
def seed_all(seed):
    
    ''' A function to seed everything for getting stable results'''
    
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = str(seed)
    os.environ['TF_KERAS'] = str(seed)
    tf.random.set_seed(seed)
    
seed_all(42)

In [ ]:
DEVICE = 'TPU'
MIXED_PRECISION = True
XLA_ACCELERATE = True

# We set our TPU settings here mixed_precision let's us bigger batches

if DEVICE == 'TPU':
    print('Connecting to TPU...')
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print('Could not connect to TPU')
        tpu = None

    if tpu:
        try:
            print('initializing  TPU ...')
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)            
            print('TPU initialized')
            if MIXED_PRECISION:
                from tensorflow.keras.mixed_precision import experimental as mixed_precision
                policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
                mixed_precision.set_policy(policy)
                print('Mixed precision enabled')
            if XLA_ACCELERATE:
                    tf.config.optimizer.set_jit(True)
                    print('Accelerated Linear Algebra enabled')
                 
        except _:
            print('failed to initialize TPU')
    else:
        DEVICE = 'GPU'

if DEVICE != 'TPU':
    print('Using default strategy for CPU and single GPU')
    strategy = tf.distribute.get_strategy()

if DEVICE == 'GPU':
    print('Num GPUs Available: ', len(tf.config.experimental.list_physical_devices('GPU')))
    if MIXED_PRECISION:
        from tensorflow.keras.mixed_precision import experimental as mixed_precision
        policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
        mixed_precision.set_policy(policy)
        print('Mixed precision enabled')
    if XLA_ACCELERATE:
        tf.config.optimizer.set_jit(True)
        print('Accelerated Linear Algebra enabled')
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

# Config

In [ ]:
train_fold = [[1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44],
 [0,
  1,
  2,
  3,
  4,
  6,
  7,
  9,
  10,
  11,
  12,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44],
 [0,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44],
 [0,
  1,
  2,
  3,
  5,
  6,
  8,
  9,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44],
 [0,
  1,
  2,
  4,
  5,
  7,
  8,
  9,
  10,
  11,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44]]
val_fold =  [[0, 9, 11], [5,  8, 13], [1,  2, 14], [4,  7, 10], [3, 6,12]]

In [ ]:
# Some modelling and augmentation parameters

CFG = dict(
    epochs = 20,
    batch_size = 256,
#     lr = 0.00016,  # learning rate 
    lr_start=0.000005,
    lr_max=0.00000125,
    lr_min=0.000001,
    lr_rampup=5,
    lr_sustain=0,
    lr_decay=0.8,
    
    eff_B = 7, #effnet to choose eg. B1-B2-...-B7
    
    sprinkles_mode    = 'normal',
    sprinkles_prob    =   1, # probability to spawn a box (between 0-1)
)


In [ ]:
inp_size = [224, 240, 260, 300, 380, 456, 528, 600]
input_size = [256, 256, 256, 384, 384, 512, 512, 768]
lr = [0.00016, 0.00016, 0.00016, 0.00016, 0.0001, 0.0001, 0.0001, 0.0001]
CFG['inp_size'] = inp_size[CFG['eff_B']]
CFG['input_size'] = input_size[CFG['eff_B']]
CFG['lr'] = lr[CFG['eff_B']]
print('Input size: '+str(CFG['input_size'])+', '+str(CFG['inp_size'])+', '+str(CFG['lr']))

# Loading Data

In [ ]:
BASEPATH = '../input/siim-isic-melanoma-classification'
df_train = pd.read_csv(os.path.join(BASEPATH, 'train.csv'))
df_test  = pd.read_csv(os.path.join(BASEPATH, 'test.csv'))
df_sub   = pd.read_csv(os.path.join(BASEPATH, 'sample_submission.csv'))

GCS_PATH = KaggleDatasets().get_gcs_path('melanoma-%ix%i'%(CFG['input_size'],CFG['input_size'])) # main train data
GCS_PATH2 = KaggleDatasets().get_gcs_path('isic2019-%ix%i'%(CFG['input_size'],CFG['input_size'])) # external data


# Training files directory
training_files_1 = tf.io.gfile.glob(GCS_PATH + '/train%.2d*.tfrec'%i for i in range(0, 15))
training_files_2019 = tf.io.gfile.glob(GCS_PATH2 + '/train%.2d*.tfrec'%i for i in range(0, 30))
training_files = training_files_1 + training_files_2019


# test files 
test_files = tf.io.gfile.glob(GCS_PATH + '/test*.tfrec') # test data

# Augmentations

In [ ]:
# Progressive sprinkles implementation from here:
# https://www.kaggle.com/benboren/tfrecord-progressive-sprinkles

def make_mask(num_holes,side_length,rows, cols, num_channels):
        '''Builds the mask for all sprinkles.'''
        row_range = tf.tile(tf.range(rows)[..., tf.newaxis], [1, num_holes])
        col_range = tf.tile(tf.range(cols)[..., tf.newaxis], [1, num_holes])
        r_idx = tf.random.uniform([num_holes], minval=0, maxval=rows-1,
                                  dtype=tf.int32)
        c_idx = tf.random.uniform([num_holes], minval=0, maxval=cols-1,
                                  dtype=tf.int32)
        r1 = tf.clip_by_value(r_idx - side_length // 2, 0, rows)
        r2 = tf.clip_by_value(r_idx + side_length // 2, 0, rows)
        c1 = tf.clip_by_value(c_idx - side_length // 2, 0, cols)
        c2 = tf.clip_by_value(c_idx + side_length // 2, 0, cols)
        row_mask = (row_range > r1) & (row_range < r2)
        col_mask = (col_range > c1) & (col_range < c2)

        # Combine masks into one layer and duplicate over channels.
        mask = row_mask[:, tf.newaxis] & col_mask
        mask = tf.reduce_any(mask, axis=-1)
        mask = mask[..., tf.newaxis]
        mask = tf.tile(mask, [1, 1, num_channels])
        return mask
    
def sprinkles(image, cfg = CFG): 
    
    '''Applies all sprinkles.'''
    
    num_holes = cfg['num_holes']
    side_length = cfg['side_length']
    mode = cfg['sprinkles_mode']
    PROBABILITY = cfg['sprinkles_prob']
    
    RandProb = tf.cast( tf.random.uniform([],0,1) < PROBABILITY, tf.int32)
    if (RandProb == 0)|(num_holes == 0): return image
    
    img_shape = tf.shape(image)
    if mode is 'normal':
        rejected = tf.zeros_like(image)
    elif mode is 'salt_pepper':
        num_holes = num_holes // 2
        rejected_high = tf.ones_like(image)
        rejected_low = tf.zeros_like(image)
    elif mode is 'gaussian':
        rejected = tf.random.normal(img_shape, dtype=tf.float32)
    else:
        raise ValueError(f'Unknown mode "{mode}" given.')
        
    rows = img_shape[0]
    cols = img_shape[1]
    num_channels = img_shape[-1]
    if mode is 'salt_pepper':
        mask1 = make_mask(num_holes,side_length,rows, cols, num_channels)
        mask2 = make_mask(num_holes,side_length,rows, cols, num_channels)
        filtered_image = tf.where(mask1, rejected_high, image)
        filtered_image = tf.where(mask2, rejected_low, filtered_image)
    else:
        mask = make_mask(num_holes,side_length,rows, cols, num_channels)
        filtered_image = tf.where(mask, rejected, image)
    return filtered_image

def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    
    # Most of the augmentations and transforms from here:
    # https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))

def transform(image, label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = CFG['inp_size']
    XDIM = DIM%2 #fix for size 331
    
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        rot = 15. * tf.random.normal([1],dtype='float32')
    else:
        rot = 180. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image['img_inp'],tf.transpose(idx3))
        
    return {'img_inp': tf.reshape(d,[DIM,DIM,3]), 'meta_inp': image['meta_inp']}, label

def decode_image(image_data):
    
    '''A function for loading image, decode and reshape'''
    
    image = tf.image.decode_jpeg(image_data, channels=3)
    # normalizing image
    
#     image = tf.cast(image, tf.float32) / 255.0 
#     image = tf.decode_raw(image, tf.uint8) / 255.0 

    
    # explicit input size for TPU
    image = tf.image.resize(image, [CFG['inp_size'], CFG['inp_size']])
    image = tf.reshape(image, [CFG['inp_size'], CFG['inp_size'], 3])
#     image = tf.image.rgb_to_hsv(image)
    image = tf.cast(image, tf.float32) / 255.0
#     image = tf.image.resize(image, [CFG['inp_size'], CFG['inp_size']])
    return image

def data_augment(data, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement 
    # in the next function (below), this happens essentially for free on TPU. 
    # Data pipeline code is executed on the 'CPU' part
    # of the TPU while the TPU itself is computing gradients.
    # https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96
#     data['img_inp'] = tf.image.resize(data['img_inp'], [CFG['inp_size'], CFG['inp_size']])
    data['img_inp'] = tf.image.random_flip_left_right(data['img_inp'])
    data['img_inp'] = tf.image.random_flip_up_down(data['img_inp'])
    data['img_inp'] = tf.image.random_hue(data['img_inp'], 0.01)
    data['img_inp'] = tf.image.random_saturation(data['img_inp'], 0.7, 1.3)
    data['img_inp'] = tf.image.random_contrast(data['img_inp'], 0.8, 1.2)
    data['img_inp'] = tf.image.random_brightness(data['img_inp'], 0.1)
    data['img_inp'] = sprinkles(data['img_inp']) 

    
    
    return data, label

# Read Data

In [ ]:
def read_labeled_tfrecord(example):
    
    '''A function to parse images and returns targets together'''
    
    LABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        'image': tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        'target': tf.io.FixedLenFeature([], tf.int64),
        # meta features
        'age_approx': tf.io.FixedLenFeature([], tf.int64),
        'sex': tf.io.FixedLenFeature([], tf.int64),
        'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64)
        
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
#     image = tf.image.resize(decode_image(example['image']), [224, 224])
    label = tf.cast(example['target'], tf.int32)
    # dictionary for meta images
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    # returns a dataset of (image, label, data)
    return image, label, data

def read_unlabeled_tfrecord(example):
    
    '''A function to parse images and returns image ids together'''
    
    UNLABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        'image': tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        'image_name': tf.io.FixedLenFeature([], tf.string),
        # meta features
        'age_approx': tf.io.FixedLenFeature([], tf.int64),
        'sex': tf.io.FixedLenFeature([], tf.int64),
        'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
#     image = tf.image.resize(decode_image(example['image']), [224, 224])
    image_name = example['image_name']
    # dictionary for meta images
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    # returns a dataset of (image, key, data)
    return image, image_name, data

def read_complete_tfrecord(example):
    
    '''A function to parse images and returns image ids as well as targets together'''
    
    LABELED_TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string), 
        'image_name': tf.io.FixedLenFeature([], tf.string), 
        'target': tf.io.FixedLenFeature([], tf.int64), 
        # meta features
        'age_approx': tf.io.FixedLenFeature([], tf.int64),
        'sex': tf.io.FixedLenFeature([], tf.int64),
        'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
#     image = tf.image.resize(decode_image(example['image']), [224, 224])
    image_name = example['image_name']
    target = tf.cast(example['target'], tf.int32)
    # dictionary for meta images
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    return image, image_name, target, data

def load_dataset(filenames, labeled = True, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        # disable order, increase speed
        ignore_order.experimental_deterministic = False 
        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # use data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO) 
    return dataset

def load_complete_dataset(filenames):        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # returns a dataset of (image_name, target)
    dataset = dataset.map(read_complete_tfrecord, num_parallel_calls = AUTO) 
    return dataset

def training_input(image, label, data):
    
    ''' A function for extracing metadata as well as images for train'''
    
    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'img_inp': image, 'meta_inp':  tabular}, label

def test_input(image, image_name, data):
    
    ''' A function for extracing metadata as well as images for test'''
    

    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
   
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'img_inp': image, 'meta_inp':  tabular}, image_name

def validation_input(image, image_name, target, data):
    
    ''' A function for extracing metadata as well as images for validation'''    

    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'img_inp': image, 'meta_inp':  tabular}, image_name, target

def get_training_dataset(filenames, labeled = True, ordered = False):
    
    '''Gets the data for training phrase, applies augment, transformation and shuffle'''
    
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(training_input, num_parallel_calls = AUTO)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.map(transform, num_parallel_calls = AUTO)
    # the training dataset must repeat for several epochs
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(CFG['batch_size'])
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)
    return dataset


def get_validation_dataset(filenames, labeled = True, ordered = True):
    
    '''Gets the data for validation phrase'''
    
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(training_input, num_parallel_calls = AUTO)
    dataset = dataset.batch(CFG['batch_size'])
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_test_dataset(filenames, labeled = False, ordered = True):
    
    '''Gets the data for testing phrase, no augmentations since no TTA'''
    
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(test_input, num_parallel_calls = AUTO)
    dataset = dataset.batch(CFG['batch_size'])
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_complete_dataset(filenames):
    dataset = load_complete_dataset(filenames)
    dataset = dataset.map(validation_input, num_parallel_calls = AUTO)
    dataset = dataset.batch(CFG['batch_size'])
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    
    ''' The number of data items is written in the name of the .tfrec files '''
    
    n = [int(re.compile(r'-([0-9]*)\.').search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
NUM_TRAINING_IMAGES = int(count_data_items(training_files_1) * 0.8 + count_data_items(training_files_2019))
# use validation data for training
NUM_VALIDATION_IMAGES = int(count_data_items(training_files_1) * 0.2)
NUM_TEST_IMAGES = count_data_items(test_files)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // CFG['batch_size']

print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

# Custom Sprinkles

In [ ]:
class IncreaseSprinklesHoles(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        
        ''' Custom callback for progressive sprinkles increases with epoch'''
      
        if epoch <= 10:
            CFG['num_holes'] = epoch  + 5
            CFG['side_length'] = (CFG['inp_size'] // 10) + (epoch // 2)
        if epoch >= 10:
            CFG['num_holes'] = epoch  + 5
            CFG['side_length'] = (CFG['inp_size'] // 5) + (epoch // 2)
        if epoch >= 15:
            CFG['num_holes'] = epoch  + 5
            CFG['side_length'] = (CFG['inp_size'] // 2) + (epoch // 2)
        
sprinkles_cb = IncreaseSprinklesHoles()

# Modelling and OOF Predictions

In [ ]:
def get_model():
    
    ''' Main modelling part with effnet, metadata, effnets, callbacks, optimizers etc.'''
    
    with strategy.scope():
        
        # meta implementation from here:
        # https://www.kaggle.com/rajnishe/rc-fork-siim-isic-melanoma-384x384/notebook
        
        img_inp = tf.keras.layers.Input(shape = (CFG['inp_size'], CFG['inp_size'], 3), name = 'img_inp')
        meta_inp = tf.keras.layers.Input(shape = (9), name = 'meta_inp')
        
        effs = [0,1,2,3,4,5,6,7]
        eff = effs[CFG['eff_B']]
        
        constructor = getattr(efn, f'EfficientNetB{eff}')
        efnetb = constructor(weights = 'noisy-student', include_top = False)     
        
        # attention implementation from here:
        # https://www.kaggle.com/kmader/attention-on-pretrained-vgg16-for-bone-age/notebook
        
        pt_depth = efnetb.get_output_shape_at(0)[-1]
        pt_features = efnetb(img_inp)
        bn_features = tf.keras.layers.BatchNormalization()(pt_features)
        
        # here we do an attention mechanism to turn pixels in the GAP on an off
        attn_layer = tf.keras.layers.Conv2D(64, kernel_size = (1, 1), padding = 'same', activation = 'swish')(tf.keras.layers.GaussianDropout(0.5)(bn_features))
        attn_layer = tf.keras.layers.Conv2D(16, kernel_size = (1, 1), padding = 'same', activation = 'swish')(attn_layer)
        attn_layer = tf.keras.layers.Conv2D(8, kernel_size = (1,1), padding = 'same', activation = 'swish')(attn_layer)
        attn_layer = tf.keras.layers.Conv2D(1, kernel_size = (1, 1), padding = 'valid', activation = 'sigmoid')(attn_layer)
        
        # fan it out to all of the channels
        up_c2_w = np.ones((1, 1, 1, pt_depth))
        up_c2 = tf.keras.layers.Conv2D(pt_depth, kernel_size = (1, 1), padding = 'same',  activation = 'linear',  use_bias = False,    weights = [up_c2_w]  )
        up_c2.trainable = False
        attn_layer = up_c2(attn_layer)
        mask_features = tf.keras.layers.multiply([attn_layer, bn_features])
        gap_features = tf.keras.layers.GlobalAveragePooling2D()(mask_features)
        gap_mask = tf.keras.layers.GlobalAveragePooling2D()(attn_layer)
        
         # To account for missing values from the attention model
        gap = tf.keras.layers.Lambda(lambda x: x[0] / x[1], name = 'RescaleGAP')([gap_features, gap_mask])
        gap_dr = tf.keras.layers.GaussianDropout(0.5)(gap)
        dr_steps = tf.keras.layers.GaussianDropout(0.25)(tf.keras.layers.Dense(128, activation = 'swish')(gap_dr))
        
        
        meta_layer = tf.keras.layers.Dense(8)(meta_inp)
        meta_layer = tf.keras.layers.BatchNormalization()(meta_layer)
        meta_layer = tf.keras.layers.Activation('swish')(meta_layer)
        meta_layer = tf.keras.layers.GaussianDropout(0.2)(meta_layer)
        
        concat = tf.keras.layers.concatenate([dr_steps, meta_layer])
        concat = tf.keras.layers.BatchNormalization()(concat)
        concat = tf.keras.layers.Dense(512, activation = 'swish')(concat)        
        output = tf.keras.layers.Dense(1, activation = 'sigmoid',dtype='float32')(concat)

        model = tf.keras.models.Model(inputs = [img_inp, meta_inp], outputs = [output])
        
        model.summary()
                
            
        opt = tf.keras.optimizers.Adam(learning_rate = CFG['lr'])
        
        
        model.compile(
            optimizer = opt,
            loss = [tfa.losses.SigmoidFocalCrossEntropy(reduction=tf.keras.losses.Reduction.AUTO,gamma = 2.0, alpha = 0.90)],
            metrics = [tf.keras.metrics.AUC()]
        )

        return model

In [ ]:
CFG['num_holes'] = 5 
CFG['side_length'] = CFG['inp_size']//10

trn_ind = train_fold[2]
val_ind = val_fold[2]

train_dataset = get_training_dataset([training_files[x] for x in trn_ind], labeled = True, ordered = False)
val_dataset = get_validation_dataset([training_files[x] for x in val_ind], labeled = True, ordered = True)

K.clear_session()

model = get_model()
# early stopping with 5 patience
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_auc', mode = 'max', patience = 5, 
                                              verbose = 2, min_delta = 0.00025, restore_best_weights = True)

# lr scheduler with 2 patience
cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_auc', factor = 0.6, patience = 2 , verbose = 2, min_delta = 0.0005, min_lr=0.000001, mode = 'max')

# saving best model weights
cpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                    'best_res.h5', monitor='val_auc', verbose=1, save_best_only=True,
                     mode='max', save_freq='epoch')
history = model.fit(train_dataset, 
                    steps_per_epoch = STEPS_PER_EPOCH,
                    epochs = CFG['epochs'],
                    callbacks = [sprinkles_cb, cb_lr_schedule, early_stopping],
                    validation_data = val_dataset,
                    verbose = 1)

In [ ]:
# since we are splitting the dataset and iterating separately on images and ids, order matters.
test_ds = get_test_dataset(test_files, labeled = False, ordered = True)
test_images_ds = test_ds.map(lambda image, image_name: image)

print('Computing predictions...')
probabilities = np.average([np.concatenate(model.predict(test_images_ds))], axis = 0)
print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, image_name: image_name).unbatch()

test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') 
pred_df = pd.DataFrame({'image_name': test_ids, 'target': probabilities})
df_sub.drop('target', inplace = True, axis = 1)
df_sub = df_sub.merge(pred_df, on = 'image_name')
df_sub.to_csv('sub_EfficientNetB%i_%i.csv'%(CFG['eff_B'],CFG['inp_size']), index = False)